In [2]:
import numpy as np
import sys
import pprint

if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [4]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()
pp.pprint(f'{env.nS = }')
pp.pprint(f'{env.nA = }')
pp.pprint(f'{env.action_space = }')
pp.pprint(f'{env.observation_space = }')
pp.pprint(f'{env.shape = }')
pp.pprint(f'{env.reward_range = }')
pp.pprint(f'{env.spec = }')
pp.pprint(f'{env.P[0] = }')


'env.nS = np.int64(16)'
'env.nA = 4'
'env.action_space = Discrete(4)'
'env.observation_space = Discrete(16)'
'env.shape = [4, 4]'
'env.reward_range = (-inf, inf)'
'env.spec = None'
('env.P[0] = {0: [(1.0, 0, 0.0, True)], 1: [(1.0, 0, 0.0, True)], 2: [(1.0, 0, '
 '0.0, True)], 3: [(1.0, 0, 0.0, True)]}')


In [5]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    V(s)=s′∑​P(s′∣s,π(s))[R(s,π(s),s′)+γV(s′)]
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    iteration = 0
    while True:
        iteration += 1
        diff = 0
        # for each action, do full backup
        for s in range(env.nS):
            current_v = 0
            # for all possible next actions
            for a, action_prob in enumerate(policy[s]):
                # for each action, look at the reward and possible next state
                for prob, ns, r, done in env.P[s][a]:
                    # calculate the expected value
                    current_v += action_prob * prob * (r + discount_factor * V[ns])
            # check the max val change across all states
            diff = max(diff, np.abs(current_v - V[s]))        
            V[s] = current_v
        # make sure the max val change is smaller than theta before we stop
        if(diff < theta):
            break
    pp.pprint(np.array(V))
    print(iteration)
    return np.array(V)

In [6]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
pp.pprint(f'{random_policy = }')
v = policy_eval(random_policy, env, 1.0, 0.001)

('random_policy = array([[0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25],\n'
 '       [0.25, 0.25, 0.25, 0.25]])')
array([  0.        , -13.99330608, -19.99037659, -21.98940765,
       -13.99330608, -17.99178568, -19.99108113, -19.99118312,
       -19.99037659, -19.99108113, -17.99247411, -13.99438108,
       -21.98940765, -19.99118312, -13.99438108,   0.        ])
88


In [70]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

In [71]:
print(v.shape)
print(env.shape)
print(v.reshape(env.shape))

(16,)
[4, 4]
[[  0.         -13.99330608 -19.99037659 -21.98940765]
 [-13.99330608 -17.99178568 -19.99108113 -19.99118312]
 [-19.99037659 -19.99108113 -17.99247411 -13.99438108]
 [-21.98940765 -19.99118312 -13.99438108   0.        ]]
